In [ ]:
!pip install mp-api
!pip install lazypredict
!pip install skipatom
!pip install optuna
!git clone https://github.com/lantunes/skipatom.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.1/670.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.0/920.0 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *
from skipatom import AtomVectors, SkipAtomInducedModel, sum_pool
from pymatgen.core import Composition
import numpy as np
from lazypredict.Supervised import LazyClassifier, LazyRegressor
import optuna
from optuna.samplers import TPESampler
from sklearn.preprocessing import StandardScaler
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
mat2vec_model = AtomVectors.load("/content/drive/MyDrive/mat2vec.dim200.model")
skipatom_model = SkipAtomInducedModel.load("/content/skipatom/data/mp_2020_10_09.dim200.model",
                                           "/content/skipatom/data/mp_2020_10_09.training.data", min_count=2e7, top_n=5)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/LiIonDB.csv')
df = df[["Compound","IC"]]
df["ICG"] = (df["IC"]>0.0001).astype(int)
df

,Compound,IC,ICG
0,La0.29Li0.12NbO3,0.00,0
1,Li0.8La0.066Zr2(PO4)3,0.00,1
2,Li0.8La0.066Zr2(PO4)3,0.00,1
3,Li0.8La0.066Zr2(PO4)3,0.00,1
4,Li0.8La0.066Zr2(PO4)3,0.00,0
...,...,...,...
815,Nd0.55Li0.34TiO3,0.00,0
816,Sm0.25Li0.25TaO3,0.00,0
817,Sm0.52Li0.38TiO2.97,0.00,1
818,Sm0.52Li0.38TiO2.97,0.00,0


In [ ]:
df_test=pd.read_csv("/content/drive/MyDrive/test.csv")
df_test = df_test[["Compound","IC"]]
df_test["ICG"] = (df_test["IC"]>0.0001).astype(int)
df_test

,Compound,IC,ICG
0,LiLa(TiO3)2,0.00,1
1,Li9.81Sn0.81P2.19S12,0.01,1
2,Li10Ge(PS6)2,0.01,1
3,Li10.35Si1.35P1.65S12,0.01,1
4,Li14ZnGe4O16,0.00,0
5,Li2Ca(NH)2,0.00,0
6,Li2Ge7O15,0.00,0
7,Li2NH,0.00,1
8,Li2S,0.00,0
9,Li13.6Si2.8S1.2O16,0.00,0


In [ ]:
#Dropping duplicates and removing common
df=df.drop_duplicates(subset=['Compound'],keep='first').reset_index(drop=True)
df=pd.concat([df,df_test])
df=df.drop_duplicates(subset=['Compound'],keep=False).reset_index(drop=True)
df

,Compound,IC,ICG
0,La0.29Li0.12NbO3,0.00,0
1,Li0.8La0.066Zr2(PO4)3,0.00,1
2,Li6.4Al0.19La3Zr2O11.8,0.00,1
3,((Li2S)0.6 (SiS2)0.4 )95 (Li4SiO4)5,0.04,1
4,((Li2S)0.6 (SiS2)0.4)95 (Li4SiO4)5,0.00,1
...,...,...,...
457,Li14Ge2V2O16,0.00,0
458,Li15Ge3V2O4,0.00,0
459,Li14.8Ge3.4W0.6O4,0.00,0
460,Li6NBr3,0.00,0


In [ ]:
def myfunc(x):
  return Composition(x)
df['comp'] = df["Compound"].apply(myfunc)

In [ ]:
df

,Compound,IC,ICG,comp
0,La0.29Li0.12NbO3,0.00,0,"(La, Li, Nb, O)"
1,Li0.8La0.066Zr2(PO4)3,0.00,1,"(Li, La, Zr, P, O)"
2,Li6.4Al0.19La3Zr2O11.8,0.00,1,"(Li, Al, La, Zr, O)"
3,((Li2S)0.6 (SiS2)0.4 )95 (Li4SiO4)5,0.04,1,"(Li, S, Si, O)"
4,((Li2S)0.6 (SiS2)0.4)95 (Li4SiO4)5,0.00,1,"(Li, S, Si, O)"
...,...,...,...,...
457,Li14Ge2V2O16,0.00,0,"(Li, Ge, V, O)"
458,Li15Ge3V2O4,0.00,0,"(Li, Ge, V, O)"
459,Li14.8Ge3.4W0.6O4,0.00,0,"(Li, Ge, W, O)"
460,Li6NBr3,0.00,0,"(Li, N, Br)"


In [ ]:
input_vectors = np.asarray([sum_pool(x, skipatom_model.dictionary, skipatom_model.vectors) for x in df['comp']])

In [ ]:
target_vectors = np.asarray(df['ICG'])

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(input_vectors, target_vectors, random_state=42, stratify=target_vectors,train_size=0.8)

In [ ]:
len(x_train)/(len(x_val)+len(x_train))

0.7987012987012987

In [ ]:
df['ICG'].value_counts()

ICG
0    246
1    216
Name: count, dtype: int64

In [ ]:
clf = LazyClassifier(verbose=None, ignore_warnings=True, custom_metric = None)
m, p = clf.fit(x_train, x_val, y_train, y_val)

 93%|█████████▎| 27/29 [00:04<00:00,  9.38it/s]

[LightGBM] [Info] Number of positive: 173, number of negative: 196
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002583 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24688
[LightGBM] [Info] Number of data points in the train set: 369, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.468835 -> initscore=-0.124823
[LightGBM] [Info] Start training from score -0.124823
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

100%|██████████| 29/29 [00:06<00:00,  4.66it/s]


In [ ]:
print(p,m)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
LabelPropagation                   0.74               0.74     0.74      0.74   
LabelSpreading                     0.74               0.74     0.74      0.74   
XGBClassifier                      0.72               0.72     0.72      0.72   
RandomForestClassifier             0.72               0.71     0.71      0.72   
PassiveAggressiveClassifier        0.72               0.71     0.71      0.72   
LGBMClassifier                     0.69               0.69     0.69      0.69   
ExtraTreesClassifier               0.69               0.69     0.69      0.69   
LogisticRegression                 0.69               0.68     0.68      0.69   
BaggingClassifier                  0.67               0.66     0.66      0.67   
AdaBoostClassifier                 0.67               0.66     0.66      0.66   
KNeighborsClassifier        

In [ ]:
def objective(trial):
    n_neighbors = trial.suggest_int("n_neighbors", 1, 50)
    gamma = trial.suggest_float("gamma", 0.01, 100, log=True)
    model = sklearn.semi_supervised.LabelPropagation(n_neighbors=n_neighbors, gamma=gamma)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_val)
    return accuracy_score(y_val, y_pred)

In [ ]:
sampler = TPESampler(seed=1)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=500)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-04-28 13:14:38,203] A new study created in memory with name: no-name-84654dda-35bb-4e8d-b7cf-33fcedf252a3
[I 2024-04-28 13:14:38,259] Trial 0 finished with value: 0.6989247311827957 and parameters: {'n_neighbors': 21, 'gamma': 7.608481233714797}. Best is trial 0 with value: 0.6989247311827957.
[I 2024-04-28 13:14:38,289] Trial 1 finished with value: 0.7096774193548387 and parameters: {'n_neighbors': 1, 'gamma': 0.16193110912440742}. Best is trial 1 with value: 0.7096774193548387.
[I 2024-04-28 13:14:38,322] Trial 2 finished with value: 0.6021505376344086 and parameters: {'n_neighbors': 8, 'gamma': 0.02340746480576752}. Best is trial 1 with value: 0.7096774193548387.
[I 2024-04-28 13:14:38,356] Trial 3 finished with value: 0.7096774193548387 and parameters: {'n_neighbors': 10, 'gamma': 0.24112537061475747}. Best is trial 1 with value: 0.7096774193548387.
[I 2024-04-28 13:14:38,394] Trial 4 finished with value: 0.7204301075268817 and parameters: {'n_neighbors': 20, 'gamma': 1.429

Number of finished trials:  500
Best trial:
  Value:  0.7311827956989247
  Params: 
    n_neighbors: 18
    gamma: 2.113333547450854


In [ ]:
model=sklearn.semi_supervised.LabelPropagation(n_neighbors=trial.params['n_neighbors'],gamma=trial.params['gamma'])
model.fit(x_train,y_train)

LabelPropagation(gamma=2.113333547450854, n_neighbors=18)

In [ ]:
pred_train=model.predict(x_train)
pred_val=model.predict(x_val)
print(classification_report(y_train,pred_train))
print(f1_score(y_train, pred_train))
print(classification_report(y_val,pred_val))
print(f1_score(y_val, pred_val))

              precision    recall  f1-score   support

           0       0.96      0.93      0.95       196
           1       0.93      0.96      0.94       173

    accuracy                           0.95       369
   macro avg       0.95      0.95      0.95       369
weighted avg       0.95      0.95      0.95       369

0.9431818181818181
              precision    recall  f1-score   support

           0       0.78      0.70      0.74        50
           1       0.69      0.77      0.73        43

    accuracy                           0.73        93
   macro avg       0.73      0.73      0.73        93
weighted avg       0.74      0.73      0.73        93

0.7252747252747254


In [ ]:
df_test

,Compound,IC,ICG
0,LiLa(TiO3)2,0.00,1
1,Li9.81Sn0.81P2.19S12,0.01,1
2,Li10Ge(PS6)2,0.01,1
3,Li10.35Si1.35P1.65S12,0.01,1
4,Li14ZnGe4O16,0.00,0
5,Li2Ca(NH)2,0.00,0
6,Li2Ge7O15,0.00,0
7,Li2NH,0.00,1
8,Li2S,0.00,0
9,Li13.6Si2.8S1.2O16,0.00,0


In [ ]:
df_test['ICG'].value_counts()

ICG
0    29
1    10
Name: count, dtype: int64

In [ ]:
df_test["comp"]=df_test["Compound"].apply(myfunc)

In [ ]:
df_test

,Compound,IC,ICG,comp
0,LiLa(TiO3)2,0.00,1,"(Li, La, Ti, O)"
1,Li9.81Sn0.81P2.19S12,0.01,1,"(Li, Sn, P, S)"
2,Li10Ge(PS6)2,0.01,1,"(Li, Ge, P, S)"
3,Li10.35Si1.35P1.65S12,0.01,1,"(Li, Si, P, S)"
4,Li14ZnGe4O16,0.00,0,"(Li, Zn, Ge, O)"
5,Li2Ca(NH)2,0.00,0,"(Li, Ca, N, H)"
6,Li2Ge7O15,0.00,0,"(Li, Ge, O)"
7,Li2NH,0.00,1,"(Li, N, H)"
8,Li2S,0.00,0,"(Li, S)"
9,Li13.6Si2.8S1.2O16,0.00,0,"(Li, Si, S, O)"


In [ ]:
x_test= np.asarray([sum_pool(x, skipatom_model.dictionary, skipatom_model.vectors) for x in df_test['comp']])
y_test= np.asarray(df_test['ICG'])

In [ ]:
x_test

array([[ 0.45524529,  0.09582271, -0.17086297, ..., -0.06782909,
        -0.10074657,  0.04969501],
       [ 0.94878376,  0.16817218, -1.16071248, ...,  0.15836526,
        -0.01453432,  0.30427551],
       [ 0.94975597,  0.18891199, -1.17657173, ...,  0.0865007 ,
        -0.05709515,  0.26972932],
       ...,
       [ 0.04177488, -0.0667825 , -0.12532055, ..., -0.00790259,
         0.09152979,  0.09911391],
       [ 0.21491316, -0.06613202, -0.3671931 , ..., -0.02198138,
         0.0985107 , -0.08981407],
       [ 0.82144141,  0.24609442, -0.85134375, ...,  0.02528951,
        -0.25400192, -0.53032655]])

In [ ]:
pred_test=model.predict(x_test)

In [ ]:
pred_test

array([0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0])

In [ ]:
pred_test=model.predict(x_test)
print(classification_report(y_test,pred_test))

              precision    recall  f1-score   support

           0       0.88      0.79      0.84        29
           1       0.54      0.70      0.61        10

    accuracy                           0.77        39
   macro avg       0.71      0.75      0.72        39
weighted avg       0.80      0.77      0.78        39



In [ ]:
f1_score(y_test, pred_test)

0.608695652173913

In [ ]:
f1_score(y_test, pred_test, average="macro")

0.7225296442687746

In [ ]:
f1_score(y_test, pred_test, average="micro")

0.7692307692307693

In [ ]:
f1_score(y_test, pred_test, average="weighted")

0.7779872301611431

In [ ]:
df_test["preds"]=pred_test

In [ ]:
df_test

,Compound,IC,ICG,comp,preds
0,LiLa(TiO3)2,0.00,1,"(Li, La, Ti, O)",0
1,Li9.81Sn0.81P2.19S12,0.01,1,"(Li, Sn, P, S)",1
2,Li10Ge(PS6)2,0.01,1,"(Li, Ge, P, S)",1
3,Li10.35Si1.35P1.65S12,0.01,1,"(Li, Si, P, S)",1
4,Li14ZnGe4O16,0.00,0,"(Li, Zn, Ge, O)",0
5,Li2Ca(NH)2,0.00,0,"(Li, Ca, N, H)",0
6,Li2Ge7O15,0.00,0,"(Li, Ge, O)",0
7,Li2NH,0.00,1,"(Li, N, H)",0
8,Li2S,0.00,0,"(Li, S)",0
9,Li13.6Si2.8S1.2O16,0.00,0,"(Li, Si, S, O)",0


In [ ]:
df_label=pd.read_csv("/content/drive/MyDrive/test_elem_new.csv")
df_label["comp"]=df_label["Compounds"].apply(myfunc)
x_label= np.asarray([sum_pool(x, mat2vec_model.dictionary, mat2vec_model.vectors) for x in df_label['comp']])
pred_test=model.predict(x_label)
df_label["preds"]=pred_test
df_label.to_excel("predictions_allT.xlsx")

#At Room Temperature

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/LiIonDB.csv')
df = df[(df['T'] >= 22.0) & (df['T'] <= 26.0)].reset_index(drop=True)
df = df[["Compound","IC"]]
df["ICG"] = (df["IC"]>0.0001).astype(int)
df

,Compound,IC,ICG
0,La0.29Li0.12NbO3,0.00,0
1,Li0.8La0.066Zr2(PO4)3,0.00,0
2,Li6.4Al0.19La3Zr2O11.8,0.00,1
3,((Li2S)0.6 (SiS2)0.4 )95 (Li4SiO4)5,0.01,1
4,(Li2O)16.6 (SiO2)83.4,0.00,0
...,...,...,...
348,LiZr2(PO4)3,0.00,0
349,LiZr2(PO4)3,0.00,0
350,Nd0.25Li0.25TaO3,0.00,0
351,Nd0.55Li0.34TiO3,0.00,0


In [ ]:
df_test=pd.read_csv("/content/drive/MyDrive/test.csv")
df_test = df_test[["Compound","IC"]]
df_test["ICG"] = (df_test["IC"]>0.0001).astype(int)
df_test

,Compound,IC,ICG
0,LiLa(TiO3)2,0.00,1
1,Li9.81Sn0.81P2.19S12,0.01,1
2,Li10Ge(PS6)2,0.01,1
3,Li10.35Si1.35P1.65S12,0.01,1
4,Li14ZnGe4O16,0.00,0
5,Li2Ca(NH)2,0.00,0
6,Li2Ge7O15,0.00,0
7,Li2NH,0.00,1
8,Li2S,0.00,0
9,Li13.6Si2.8S1.2O16,0.00,0


In [ ]:
#Dropping duplicates and removing common
df=df.drop_duplicates(subset=['Compound'],keep='first').reset_index(drop=True)
df=pd.concat([df,df_test])
df=df.drop_duplicates(subset=['Compound'],keep=False).reset_index(drop=True)
df

,Compound,IC,ICG
0,La0.29Li0.12NbO3,0.00,0
1,Li0.8La0.066Zr2(PO4)3,0.00,0
2,Li6.4Al0.19La3Zr2O11.8,0.00,1
3,((Li2S)0.6 (SiS2)0.4 )95 (Li4SiO4)5,0.01,1
4,(Li2O)16.6 (SiO2)83.4,0.00,0
...,...,...,...
309,Li6NBr3,0.00,0
310,LiAlSiO4,0.00,0
311,LiBH4,0.00,0
312,LiNH2,0.00,0


In [ ]:
def myfunc(x):
  return Composition(x)
df['comp'] = df["Compound"].apply(myfunc)

In [ ]:
df

,Compound,IC,ICG,comp
0,La0.29Li0.12NbO3,0.00,0,"(La, Li, Nb, O)"
1,Li0.8La0.066Zr2(PO4)3,0.00,0,"(Li, La, Zr, P, O)"
2,Li6.4Al0.19La3Zr2O11.8,0.00,1,"(Li, Al, La, Zr, O)"
3,((Li2S)0.6 (SiS2)0.4 )95 (Li4SiO4)5,0.01,1,"(Li, S, Si, O)"
4,(Li2O)16.6 (SiO2)83.4,0.00,0,"(Li, O, Si)"
...,...,...,...,...
309,Li6NBr3,0.00,0,"(Li, N, Br)"
310,LiAlSiO4,0.00,0,"(Li, Al, Si, O)"
311,LiBH4,0.00,0,"(Li, B, H)"
312,LiNH2,0.00,0,"(Li, N, H)"


In [ ]:
input_vectors = np.asarray([sum_pool(x, skipatom_model.dictionary, skipatom_model.vectors) for x in df['comp']])

In [ ]:
target_vectors = np.asarray(df['ICG'])

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(input_vectors, target_vectors, random_state=42, stratify=target_vectors,train_size=0.8)

In [ ]:
len(x_train)/(len(x_val)+len(x_train))

0.7993630573248408

In [ ]:
df['ICG'].value_counts()

ICG
0    187
1    127
Name: count, dtype: int64

In [ ]:
clf = LazyClassifier(verbose=None, ignore_warnings=True, custom_metric = None)
m, p = clf.fit(x_train, x_val, y_train, y_val)

 97%|█████████▋| 28/29 [00:03<00:00,  6.28it/s]

[LightGBM] [Info] Number of positive: 102, number of negative: 149
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001564 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16946
[LightGBM] [Info] Number of data points in the train set: 251, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.406375 -> initscore=-0.378973
[LightGBM] [Info] Start training from score -0.378973
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

100%|██████████| 29/29 [00:04<00:00,  6.82it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [ ]:
print(p,m)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
LGBMClassifier                     0.86               0.83     0.83      0.85   
XGBClassifier                      0.83               0.80     0.80      0.82   
LabelSpreading                     0.79               0.79     0.79      0.79   
LabelPropagation                   0.79               0.79     0.79      0.79   
ExtraTreesClassifier               0.79               0.78     0.78      0.79   
AdaBoostClassifier                 0.79               0.78     0.78      0.79   
SGDClassifier                      0.79               0.77     0.77      0.79   
LinearSVC                          0.75               0.73     0.73      0.75   
RidgeClassifierCV                  0.75               0.73     0.73      0.75   
KNeighborsClassifier               0.73               0.72     0.72      0.73   
RandomForestClassifier      

In [ ]:
import lightgbm as lgb

def objective(trial):
    num_leaves = trial.suggest_int("num_leaves", 20, 120)
    boosting_type = trial.suggest_categorical("boosting_type", ["gbdt","dart"])
    model = lgb.LGBMClassifier(boosting_type=boosting_type,num_leaves=num_leaves,random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_val)
    return accuracy_score(y_val, y_pred)

In [ ]:
sampler = TPESampler(seed=1)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=200)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-04-20 21:03:06,657] A new study created in memory with name: no-name-b7e23159-4624-42b7-81e8-495f960f9975


[LightGBM] [Info] Number of positive: 102, number of negative: 149
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001440 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16913
[LightGBM] [Info] Number of data points in the train set: 251, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.406375 -> initscore=-0.378973
[LightGBM] [Info] Start training from score -0.378973
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

[I 2024-04-20 21:03:07,158] Trial 0 finished with value: 0.8571428571428571 and parameters: {'num_leaves': 62, 'boosting_type': 'gbdt'}. Best is trial 0 with value: 0.8571428571428571.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number 

[I 2024-04-20 21:03:07,682] Trial 1 finished with value: 0.8571428571428571 and parameters: {'num_leaves': 50, 'boosting_type': 'gbdt'}. Best is trial 0 with value: 0.8571428571428571.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 102, number of negative: 149
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001357 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16913
[LightGBM] [Info] Number of data points in the train set: 251, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.406375 -> initscore=-0.378973
[LightGBM] [Info] Start training from score -0.378973
[LightGBM] [Warning] No further splits with positive gain, b

[I 2024-04-20 21:04:23,823] Trial 198 finished with value: 0.8571428571428571 and parameters: {'num_leaves': 33, 'boosting_type': 'gbdt'}. Best is trial 0 with value: 0.8571428571428571.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-04-20 21:04:24,353] Trial 199 finished with value: 0.8571428571428571 and parameters: {'num_leaves': 68, 'boosting_type': 'gbdt'}. Best is trial 0 with value: 0.8571428571428571.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Number of finished trials:  200
Best trial:
  Value:  0.8571428571428571
  Params: 
    num_leaves: 62
    boosting_type: gbdt


In [ ]:
# model = sklearn.ensemble.AdaBoostClassifier()
# model=sklearn.ensemble.RandomForestClassifier()
# model=sklearn.semi_supervised.LabelPropagation(n_neighbors=trial.params['n_neighbors'],gamma=trial.params['gamma'])
# model = sklearn.svm.LinearSVC(C=trial.params['C'],random_state=42)
model = lgb.LGBMClassifier(boosting_type=trial.params['boosting_type'],num_leaves=trial.params['num_leaves'],random_state=42)
model.fit(x_train,y_train)

[LightGBM] [Info] Number of positive: 102, number of negative: 149
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001445 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16913
[LightGBM] [Info] Number of data points in the train set: 251, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.406375 -> initscore=-0.378973
[LightGBM] [Info] Start training from score -0.378973
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

LGBMClassifier(num_leaves=62, random_state=42)

In [ ]:
pred_train=model.predict(x_train)
pred_val=model.predict(x_val)
print(classification_report(y_train,pred_train))
print(f1_score(y_train, pred_train))
print(classification_report(y_val,pred_val))
print(f1_score(y_val, pred_val))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       149
           1       1.00      1.00      1.00       102

    accuracy                           1.00       251
   macro avg       1.00      1.00      1.00       251
weighted avg       1.00      1.00      1.00       251

1.0
              precision    recall  f1-score   support

           0       0.85      0.92      0.89        38
           1       0.86      0.76      0.81        25

    accuracy                           0.86        63
   macro avg       0.86      0.84      0.85        63
weighted avg       0.86      0.86      0.86        63

0.8085106382978724


In [ ]:
df_test

,Compound,IC,ICG
0,LiLa(TiO3)2,0.00,1
1,Li9.81Sn0.81P2.19S12,0.01,1
2,Li10Ge(PS6)2,0.01,1
3,Li10.35Si1.35P1.65S12,0.01,1
4,Li14ZnGe4O16,0.00,0
5,Li2Ca(NH)2,0.00,0
6,Li2Ge7O15,0.00,0
7,Li2NH,0.00,1
8,Li2S,0.00,0
9,Li13.6Si2.8S1.2O16,0.00,0


In [ ]:
df_test['ICG'].value_counts()

ICG
0    29
1    10
Name: count, dtype: int64

In [ ]:
df_test["comp"]=df_test["Compound"].apply(myfunc)

In [ ]:
df_test

,Compound,IC,ICG,comp
0,LiLa(TiO3)2,0.00,1,"(Li, La, Ti, O)"
1,Li9.81Sn0.81P2.19S12,0.01,1,"(Li, Sn, P, S)"
2,Li10Ge(PS6)2,0.01,1,"(Li, Ge, P, S)"
3,Li10.35Si1.35P1.65S12,0.01,1,"(Li, Si, P, S)"
4,Li14ZnGe4O16,0.00,0,"(Li, Zn, Ge, O)"
5,Li2Ca(NH)2,0.00,0,"(Li, Ca, N, H)"
6,Li2Ge7O15,0.00,0,"(Li, Ge, O)"
7,Li2NH,0.00,1,"(Li, N, H)"
8,Li2S,0.00,0,"(Li, S)"
9,Li13.6Si2.8S1.2O16,0.00,0,"(Li, Si, S, O)"


In [ ]:
x_test= np.asarray([sum_pool(x, skipatom_model.dictionary, skipatom_model.vectors) for x in df_test['comp']])
y_test= np.asarray(df_test['ICG'])

In [ ]:
x_test

array([[ 0.45524529,  0.09582271, -0.17086297, ..., -0.06782909,
        -0.10074657,  0.04969501],
       [ 0.94878376,  0.16817218, -1.16071248, ...,  0.15836526,
        -0.01453432,  0.30427551],
       [ 0.94975597,  0.18891199, -1.17657173, ...,  0.0865007 ,
        -0.05709515,  0.26972932],
       ...,
       [ 0.04177488, -0.0667825 , -0.12532055, ..., -0.00790259,
         0.09152979,  0.09911391],
       [ 0.21491316, -0.06613202, -0.3671931 , ..., -0.02198138,
         0.0985107 , -0.08981407],
       [ 0.82144141,  0.24609442, -0.85134375, ...,  0.02528951,
        -0.25400192, -0.53032655]])

In [ ]:
pred_test=model.predict(x_test)

In [ ]:
pred_test

array([0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0])

In [ ]:
pred_test=model.predict(x_test)
print(classification_report(y_test,pred_test))

              precision    recall  f1-score   support

           0       0.85      0.97      0.90        29
           1       0.83      0.50      0.62        10

    accuracy                           0.85        39
   macro avg       0.84      0.73      0.76        39
weighted avg       0.84      0.85      0.83        39



In [ ]:
df_test["preds"]=pred_test

In [ ]:
df_test

,Compound,IC,ICG,comp,preds
0,LiLa(TiO3)2,0.00,1,"(Li, La, Ti, O)",0
1,Li9.81Sn0.81P2.19S12,0.01,1,"(Li, Sn, P, S)",1
2,Li10Ge(PS6)2,0.01,1,"(Li, Ge, P, S)",1
3,Li10.35Si1.35P1.65S12,0.01,1,"(Li, Si, P, S)",1
4,Li14ZnGe4O16,0.00,0,"(Li, Zn, Ge, O)",0
5,Li2Ca(NH)2,0.00,0,"(Li, Ca, N, H)",0
6,Li2Ge7O15,0.00,0,"(Li, Ge, O)",0
7,Li2NH,0.00,1,"(Li, N, H)",0
8,Li2S,0.00,0,"(Li, S)",0
9,Li13.6Si2.8S1.2O16,0.00,0,"(Li, Si, S, O)",0


In [ ]:
df_label=pd.read_csv("/content/drive/MyDrive/test_elem_new.csv")
df_label["comp"]=df_label["Compounds"].apply(myfunc)
x_label= np.asarray([sum_pool(x, skipatom_model.dictionary, skipatom_model.vectors) for x in df_label['comp']])
pred_test=model.predict(x_label)
df_label["preds"]=pred_test
df_label.to_excel("predictions_RoomT.xlsx")

#Final Testing

EXP DATASET (LOOCV)

In [ ]:
df_test

,Compound,IC,ICG,comp,preds
0,LiLa(TiO3)2,0.00,1,"(Li, La, Ti, O)",0
1,Li9.81Sn0.81P2.19S12,0.01,1,"(Li, Sn, P, S)",1
2,Li10Ge(PS6)2,0.01,1,"(Li, Ge, P, S)",1
3,Li10.35Si1.35P1.65S12,0.01,1,"(Li, Si, P, S)",1
4,Li14ZnGe4O16,0.00,0,"(Li, Zn, Ge, O)",0
5,Li2Ca(NH)2,0.00,0,"(Li, Ca, N, H)",0
6,Li2Ge7O15,0.00,0,"(Li, Ge, O)",0
7,Li2NH,0.00,1,"(Li, N, H)",0
8,Li2S,0.00,0,"(Li, S)",0
9,Li13.6Si2.8S1.2O16,0.00,0,"(Li, Si, S, O)",0


In [ ]:
df_results=df_test
x_label= np.asarray([sum_pool(x, skipatom_model.dictionary, skipatom_model.vectors) for x in df_results['comp']])
pred_test=model.predict(x_label)
y_label= np.asarray(df_results["ICG"])

In [ ]:
print(classification_report(y_label,pred_test))

              precision    recall  f1-score   support

           0       0.85      0.97      0.90        29
           1       0.83      0.50      0.62        10

    accuracy                           0.85        39
   macro avg       0.84      0.73      0.76        39
weighted avg       0.84      0.85      0.83        39



In [ ]:
X, y = x_label,y_label
# print(X.shape, y.shape)
# create loocv procedure
cv = LeaveOneOut()
# create model
model1 = lgb.LGBMClassifier(boosting_type=trial.params['boosting_type'],num_leaves=trial.params['num_leaves'],random_state=42)
# evaluate model
scores = cross_val_score(model1, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (STD: %.3f)' % (mean(scores), std(scores)))

Accuracy: 0.744 (STD: 0.437)


MP DATASET

In [ ]:
df_results=pd.read_csv("/content/drive/MyDrive/MP_Dataset.csv")
df_results["ICG"]=(df_results["Probs"]>0.5).astype(int)
df_results=df_results.drop_duplicates(subset=['Compound'],keep='first').reset_index(drop=True)
df_results["comp"]=df_results["Compound"].apply(myfunc)
x_label= np.asarray([sum_pool(x, skipatom_model.dictionary, skipatom_model.vectors) for x in df_results['comp']])
pred_test=model.predict(x_label)
y_label= np.asarray(df_results["ICG"])

In [ ]:
print(classification_report(y_label,pred_test))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96       499
           1       0.22      0.05      0.09        37

    accuracy                           0.92       536
   macro avg       0.58      0.52      0.52       536
weighted avg       0.88      0.92      0.90       536



In [ ]:
X, y = x_label,y_label
# print(X.shape, y.shape)
# create loocv procedure
cv = LeaveOneOut()
# create model
model1 = lgb.LGBMClassifier(boosting_type=trial.params['boosting_type'],num_leaves=trial.params['num_leaves'],random_state=42)
# evaluate model
scores = cross_val_score(model1, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (STD: %.3f)' % (mean(scores), std(scores)))

Accuracy: 0.942 (STD: 0.233)


DFT DATASET

In [ ]:
df_results=pd.read_csv("/content/drive/MyDrive/final_test.csv")
df_results=df_results[df_results["Table"]>1]
df_results["comp"]=df_results["Compound"].apply(myfunc)
x_label= np.asarray([sum_pool(x, skipatom_model.dictionary, skipatom_model.vectors) for x in df_results['comp']])
pred_test=model.predict(x_label)
y_label= np.asarray(df_results["ICG"])

In [ ]:
print(classification_report(y_label,pred_test))

              precision    recall  f1-score   support

           0       0.86      1.00      0.92        18
           1       0.00      0.00      0.00         3

    accuracy                           0.86        21
   macro avg       0.43      0.50      0.46        21
weighted avg       0.73      0.86      0.79        21



In [ ]:
X, y = x_label,y_label
# print(X.shape, y.shape)
# create loocv procedure
cv = LeaveOneOut()
# create model
model1 = lgb.LGBMClassifier(boosting_type=trial.params['boosting_type'],num_leaves=trial.params['num_leaves'],random_state=42)
# evaluate model
scores = cross_val_score(model1, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (STD: %.3f)' % (mean(scores), std(scores)))

Accuracy: 0.857 (STD: 0.350)


In [ ]:
# df_results=pd.read_csv("/content/drive/MyDrive/final_test.csv")
# df_results=df_results[df_results["Table"]<2]
# df_results=df_results.drop_duplicates(subset=['Compound'],keep='first').reset_index(drop=True)
# df_results["comp"]=df_results["Compound"].apply(myfunc)
# x_label= np.asarray([sum_pool(x, mat2vec_model.dictionary, mat2vec_model.vectors) for x in df_results['comp']])
# pred_test=model.predict(x_label)
# y_label= np.asarray(df_results["ICG"])

In [ ]:
# print(classification_report(y_label,pred_test))

In [ ]:
# df_results["preds"]=pred_test
# df_results

LLZO DATASET

In [ ]:
df_results=pd.read_csv("/content/drive/MyDrive/Varun_LLZO-DATA.csv")
df_results["comp"]=df_results["Combined"].apply(myfunc)
x_label= np.asarray([sum_pool(x, skipatom_model.dictionary, skipatom_model.vectors) for x in df_results['comp']])
pred_test=model.predict(x_label)
y_label= np.asarray(df_results["good_cond"])

In [ ]:
print(classification_report(y_label,pred_test))

              precision    recall  f1-score   support

           0       0.81      0.19      0.31        88
           1       0.54      0.95      0.69        88

    accuracy                           0.57       176
   macro avg       0.68      0.57      0.50       176
weighted avg       0.68      0.57      0.50       176



In [ ]:
X, y = x_label,y_label
# print(X.shape, y.shape)
# create loocv procedure
cv = LeaveOneOut()
# create model
model1 = lgb.LGBMClassifier(boosting_type=trial.params['boosting_type'],num_leaves=trial.params['num_leaves'],random_state=42)
# evaluate model
scores = cross_val_score(model1, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (STD: %.3f)' % (mean(scores), std(scores)))

Accuracy: 0.722 (STD: 0.448)


LLZO DATASET AT ROOM T

In [ ]:
df_results=pd.read_csv("/content/drive/MyDrive/RoomT.csv")
df_results=df_results[df_results['room_t']>0]
df_results["comp"]=df_results["Combined"].apply(myfunc)
x_label= np.asarray([sum_pool(x, skipatom_model.dictionary, skipatom_model.vectors) for x in df_results['comp']])
pred_test=model.predict(x_label)
y_label= np.asarray(df_results["good_cond"])

In [ ]:
print(classification_report(y_label,pred_test))

              precision    recall  f1-score   support

           0       0.83      0.21      0.34        70
           1       0.55      0.96      0.70        70

    accuracy                           0.59       140
   macro avg       0.69      0.59      0.52       140
weighted avg       0.69      0.59      0.52       140



In [ ]:
X, y = x_label,y_label
# print(X.shape, y.shape)
# create loocv procedure
cv = LeaveOneOut()
# create model
model1 = lgb.LGBMClassifier(boosting_type=trial.params['boosting_type'],num_leaves=trial.params['num_leaves'],random_state=42)
# evaluate model
scores = cross_val_score(model1, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (STD: %.3f)' % (mean(scores), std(scores)))

Accuracy: 0.771 (STD: 0.420)
